<a href="https://colab.research.google.com/github/zaeyonz/Wine_Classification/blob/main/Wine_Classification_(Cross_validation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Validation Set***
*   train set를 test set이외에 또 나눔 (20%~30%정도)
*   test set를 사용하지 않고 과대,과소적합인지를 측정하는 가장 간단한 방법


In [3]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()     # class 열을 타깃으로 사용, 나머지 열은 특성 배열에 저장
target = wine['class'].to_numpy()

In [4]:
# train set & test set를 나눔
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42)

In [5]:
# train_input과 train_target을 다시 훈련 세트 sub_input, sub_target과 val_input, val_target으로 나눔

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state=42)

In [7]:
# train set와 validation set 의 크기 확인

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


*   ***원래 5197개였던 train set가 4157개로 줄고, validation set는 1040개가 되었음***

# ***Model Training***
*   sub_input, sub_target, val_input, val_target을 사용해 model을 만들고 평가

In [8]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


*   ***확실히 train set에 과대적합되어 있음***
*   매개변수를 바꿔가며 더 좋은 모델을 찾아야함

# ***Cross Validation (교차 검증)***
*   교차 검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있음
*   검증 세트를 떼어 내어 평가하는 과정을 여러번 반복함
*   보통 5-폴드, 10-폴드 교차 검증을 많이 사용함 --> 이렇게 하면 데이터의 80 ~ 90%까지 훈련에 사용 가능
*   ***scikit-learn에는 cross_validation()이라는 교차 검증 함수가 있음***

In [10]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.03412437, 0.02872252, 0.02057195, 0.01842117, 0.01073933]), 'score_time': array([0.01207399, 0.00184107, 0.01521111, 0.00907183, 0.00158525]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


*   fit_time: 각각의 모델을 훈련하는 시간
*   score_time: 모델을 검증 하는 시간
*   test_score: 점수

In [12]:
# 교차 검증의 최종 점수는 test_score키에 담긴 5개릐 점수를 평균하여 얻음

import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


*   ***cross_validate()는 train_set를 섞어 폴드를 나누지 않음!!***
*   앞서 train_test_split() 함수로 자동으로 전체 데이터를 섞은 후 데이터를 나눴기에 따로 섞을 필요는 없음
*   하지만 만약 교차 검증을 할 때 train set를 섞으려면 ***분할기(splitter)***를 지정해야함
---

*   cross_validate() 함수는 기본적으로 회귀 모델의 경우 KFold 분할기를 사용함
*   분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용함

In [13]:
# 앞서 수행한 교차 검증은 아래와 동일함!!

from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [14]:
# 만약 train set를 섞은 후 10-폴드 교차 검증을 수행하려면..

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)      # n_splits 매개변수는 몇(k) 폴드 교차 검증을 할지 정함
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# ***하이퍼파라미터 튜닝***
*   머신러닝 모델이 학습하는 파라미터를 모델 파라미터라고 부름
*   반면, ***모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 함***
*   먼저 라이브러리가 제공하는 기본값을 그대로 사용 후, 매개변수를 바꿔가며 모델을 훈련, 교차 검증 함
---
*   한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾으면 안됨!
*   ***여러 매개변수를 동시에 바꿔가며 최적의 값을 찾아야함!!***

# ***Grid Search (그리드 서치)***
*   많은 매개변수들을 한번에 최적의 값을 찾아주는 클래스
*   ***하이퍼파라미터 탐색과 교차 검증을 한 번에 수행함!!***
*   scikit-learn의 ***GridSearchCV()***

In [15]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.001, 0.002, 0.003, 0.004, 0.005]}         # 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듦

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)